In [1]:
import pandas as pd
import os

PATH = r'C:\Users\Никита\Андан\Project\data'

## GDP RU

In [27]:
def process_gdp_ru(path):
    df = pd.read_csv(os.path.join(path, r'macro\GDP_processed.csv'))
    def days_in_quarter(year, quarter):
        if quarter == 1:
            return 90 if year % 4 != 0 else 91  # проверка високосного года
        elif quarter == 2:
            return 91
        elif quarter == 3:
            return 92
        elif quarter == 4:
            return 92
    
    def start_of_quarter(year, quarter):
        '''
        Q1 начинается с 1 января.
        Q2 начинается с 1 апреля.
        Q3 начинается с 1 июля.
        Q4 начинается с 1 октября.
        '''
        if quarter == 1:
            return f"{year}-01-01"
        elif quarter == 2:
            return f"{year}-04-01"
        elif quarter == 3:
            return f"{year}-07-01"
        elif quarter == 4:
            return f"{year}-10-01"
    
    # Расширяем DataFrame до дневных данных
    daily_data = []
    for index, row in df.iterrows():
        num_days = days_in_quarter(row['year'], row['quater'])
        daily_gdp = row['gdp in 2021 prices'] / num_days
        daily_gdp_no_season = row['gdp without seasonality in 2021 prices'] / num_days
        start_date = start_of_quarter(int(row['year']), int(row['quater']))
        end_date = pd.Period(start_date, freq='Q').end_time.strftime('%Y-%m-%d')
        dates = pd.date_range(start=start_date, end=end_date, freq='D')
        for date in dates:
            daily_data.append({'date': date, 'daily_gdp': daily_gdp, 'daily_gdp_no_season': daily_gdp_no_season})
    
    # Создаем новый DataFrame
    return pd.DataFrame(daily_data)

## Macro RU

Универсально для macro_agg и households_bm

In [17]:
def process_macro_monthtly(path, file):
    df = pd.read_csv(os.path.join(path, f'macro\{file}'))
    def get_last_date(date):
        '''
        date: TimeStamp
        '''
        return (date+pd.DateOffset(months=1)).replace(day=1) - pd.Timedelta(days=1)
    
    daily_data = []
    for index, row in df.iterrows():
        start_date = pd.to_datetime(row['date'], format = '%Y-%m-%d')
        last_date = get_last_date(start_date)
        dates = pd.date_range(start=start_date, end=last_date, freq='D')
        
        for date in dates:
            daily_record = {'date': date}
            for col in df.columns[1:]:
                daily_record[col] = row[col]
            daily_data.append(daily_record)
    return pd.DataFrame(daily_data)

In [57]:
process_macro_monthtly(PATH, 'monetary_agg_processed.csv').head()

,date,Денежный агрегат М0,Денежный агрегат М1,Денежный агрегат М2,Широкая денежная масса
0,2001-01-01,418.9,869.4,1150.6,1573.8
1,2001-01-02,418.9,869.4,1150.6,1573.8
2,2001-01-03,418.9,869.4,1150.6,1573.8
3,2001-01-04,418.9,869.4,1150.6,1573.8
4,2001-01-05,418.9,869.4,1150.6,1573.8


In [61]:
df = pd.read_csv(os.path.join(PATH, r'macro\nonfinancial_sector_households_debt_extended_processed.csv'))

In [65]:
df.head()

,"Долг нефинансового сектора и домашних хозяйств, итого",Нефинансовый сектор,Внутренние заимствования,Кредиты,Долговые ценные бумаги в портфеле резидентов,Внешние заимствования,"Кредиты и займы, полученные от нерезидентов","Долговые ценные бумаги в портфеле нерезидентов, включая векселя",Домашние хозяйства и НКООДХ,Внутренние заимствования.1,Внешние заимствования.1,date
0,39267862,30555941,20979755,19340212,1639543,9576186,9450744,125442,8711921,8492165,219756,2013-01-01
1,41641441,32464355,21415828,19693260,1722568,11048527,10915324,133203,9177086,8905818,271268,2013-04-01
2,44286477,34347540,22429633,20657017,1772616,11917907,11790084,127823,9938937,9645301,293636,2013-07-01
3,46181943,35591204,23590436,21754043,1836393,12000768,11895100,105668,10590739,10284975,305764,2013-10-01
4,47752644,36600506,24100233,22196461,1903772,12500273,12380763,119510,11152138,10877194,274944,2014-01-01


In [18]:
def process_debt(path):
    df = pd.read_csv(os.path.join(path, r'macro\nonfinancial_sector_households_debt_extended_processed.csv'))
    def get_last_date(date):
        '''
        date: TimeStamp
        '''
        return (date+pd.DateOffset(months=3)).replace(day=1) - pd.Timedelta(days=1)
    
    # Расширяем DataFrame до дневных данных
    daily_data = []
    for index, row in df.iterrows():
        start_date = pd.to_datetime(row['date'])
        end_date = get_last_date(start_date)
        dates = pd.date_range(start=start_date, end=end_date, freq='D')
        
        for date in dates:
            daily_record = {'date': date}
            for col in df.columns[:-1]:
                daily_record[col] = row[col]
            daily_data.append(daily_record)
    
    # Создаем новый DataFrame
    return pd.DataFrame(daily_data)

In [73]:
process_debt(PATH).head(2)

,date,"Долг нефинансового сектора и домашних хозяйств, итого",Нефинансовый сектор,Внутренние заимствования,Кредиты,Долговые ценные бумаги в портфеле резидентов,Внешние заимствования,"Кредиты и займы, полученные от нерезидентов","Долговые ценные бумаги в портфеле нерезидентов, включая векселя",Домашние хозяйства и НКООДХ,Внутренние заимствования.1,Внешние заимствования.1
0,2013-01-01,39267862,30555941,20979755,19340212,1639543,9576186,9450744,125442,8711921,8492165,219756
1,2013-01-02,39267862,30555941,20979755,19340212,1639543,9576186,9450744,125442,8711921,8492165,219756


In [ ]:
# imoex поменять rename('Unnamed: 0': 'date')
pd.read_csv(os.path.join(PATH, r'macro\imoex.csv'))

In [ ]:
# тоже поменять  rename('Unnamed: 0': 'date')
pd.read_csv(os.path.join(PATH, r'macro\data_interbank_rates.csv'))

In [ ]:
# тоже поменять  rename('Unnamed: 0': 'date')
pd.read_csv(os.path.join(PATH, r'macro\data_precious_metals.csv'))

## US indicators

### GDP

In [19]:
def process_gdp_usa(path):
    df = pd.read_csv(os.path.join(path, r'macro\GDP_USA.csv'))
    def get_last_date(date):
        '''
        date: TimeStamp
        '''
        return (date+pd.DateOffset(months=3)).replace(day=1) - pd.Timedelta(days=1)
    
    # Расширяем DataFrame до дневных данных
    daily_data = []
    for index, row in df.iterrows():
        start_date = pd.to_datetime(row['date'])
        end_date = get_last_date(start_date)
        dates = pd.date_range(start=start_date, end=end_date, freq='D')
        num_days = len(dates)
        
        for date in dates:
            daily_data.append({'date': date, 'real_gdp_usa': row['value'] / num_days})
            
    
    # Создаем новый DataFrame
    return pd.DataFrame(daily_data)

In [85]:
process_gdp_usa(PATH).head()

,date,real_gdp_usa
0,2024-01-01,61.264473
1,2024-01-02,61.264473
2,2024-01-03,61.264473
3,2024-01-04,61.264473
4,2024-01-05,61.264473


### S&P500

In [13]:
# здесь единственное - переименовать столбцы и убедиться, что дата в правильном формате
df = pd.read_csv(os.path.join(PATH, r'macro\S&P500.csv'))
df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

### EFFR

In [91]:
df = pd.read_csv(os.path.join(PATH, r'macro\EFFR.csv'))

In [93]:
df.head(20)

,DATE,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83
5,1954-12-01,1.28
6,1955-01-01,1.39
7,1955-02-01,1.29
8,1955-03-01,1.35
9,1955-04-01,1.43


In [20]:
def process_effr(path):
    df = pd.read_csv(os.path.join(path, r'macro\EFFR.csv'))
    def get_last_date(date):
        '''
        date: TimeStamp
        '''
        return (date+pd.DateOffset(months=1)).replace(day=1) - pd.Timedelta(days=1)
    
    daily_data = []
    for index, row in df.iterrows():
        start_date = pd.to_datetime(row['DATE'], format = '%Y-%m-%d')
        last_date = get_last_date(start_date)
        dates = pd.date_range(start=start_date, end=last_date, freq='D')
        
        for date in dates:
            daily_data.append({'date': date, 'effr': row['FEDFUNDS']})

    return pd.DataFrame(daily_data)

In [100]:
process_effr(PATH).head()

,date,effr
0,1954-07-01,0.8
1,1954-07-02,0.8
2,1954-07-03,0.8
3,1954-07-04,0.8
4,1954-07-05,0.8


In [106]:
# переименовать .rename('Unnamed: 0': 'date')
pd.read_csv(os.path.join(PATH, r'macro\data_usdrub.csv'))

,Unnamed: 0,usdrub
0,2010-01-01,30.1851
1,2010-01-04,29.4956
2,2010-01-05,29.1537
3,2010-01-06,30.7400
4,2010-01-07,31.2554
...,...,...
3536,2024-11-01,89.3939
3537,2024-11-04,93.2198
3538,2024-12-01,88.7818
3539,2024-12-03,90.6252


## Котировки

In [110]:
# CLOSE, VOLUME, TRADEDATE
df1 = pd.read_csv(os.path.join(PATH, r'renewable_energy\IRAO_RU.csv'))

,Unnamed: 0,BOARDID,TRADEDATE,CLOSE,VOLUME,VALUE
0,0,TQBR,2013-07-08,NaN,0,0.0
1,1,TQBR,2013-07-09,NaN,0,0.0
2,2,TQBR,2013-07-10,NaN,0,0.0
3,3,TQBR,2013-07-11,0.01292,75000000,968750.0
4,4,TQBR,2013-07-12,NaN,0,0.0


In [113]:
# volume, close, date, в date нужно вычленить только день
pd.read_csv(os.path.join(PATH, r'renewable_energy\ALTO_USA.csv')).tail()

,Unnamed: 0,date,open,low,high,close,volume
157,157,2023-11-15 12:30:00,2.445,2.39,2.47,2.4000,1194975
158,158,2023-11-14 12:30:00,2.345,2.28,2.39,2.3701,944578
159,159,2023-11-13 12:30:00,2.220,2.18,2.24,2.1950,768349
160,160,2023-11-10 12:30:00,2.080,2.03,2.14,2.1100,1095663
161,161,2023-11-09 12:30:00,2.070,2.04,2.09,2.0550,1122532


In [12]:
os.listdir(os.path.join(PATH, 'renewable_energy'))[0]

'ALTO_USA.csv'

In [28]:
def merge_stocks(path, path_total):
    def merge_dfs(left, right):
        return pd.merge(left, right, on = 'date', how = 'outer')
        
    file_names = os.listdir(path)
    for indx, file_name in enumerate(file_names):
        df = pd.read_csv(os.path.join(path, file_name))
        ticker_name = file_name[:-4]
        if 'USA' == file_name[-7:-4]:
            df = df[['volume', 'close', 'date']]
            df['date'] = df['date'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d'))
            df['date'] = pd.to_datetime(df['date'])
            df[['volume', 'close']] = df[['volume', 'close']].astype('float32')
            df.rename({'volume': f'volume_{ticker_name}', \
                       'close': f'close_{ticker_name}'}, axis = 1, inplace = True)
        elif 'RU' == file_name[-6:-4]:
            df = df[['TRADEDATE', 'VOLUME', 'CLOSE']]
            df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])
            df[['VOLUME', 'CLOSE']] = df[['VOLUME', 'CLOSE']].astype('float32')
            df.rename({'TRADEDATE': 'date', \
                       'VOLUME': f'volume_{ticker_name}', \
                      'CLOSE': f'close_{ticker_name}'}, axis = 1, inplace = True)
        if indx == 0:
            final_df = df
        else:
            final_df = merge_dfs(final_df, df)
            del df

    # Переходим к макроданным
    # курс доллара
    usd_rub = pd.read_csv(os.path.join(path_total, r'macro\data_usdrub.csv'))
    usd_rub.rename({'Unnamed: 0':'date'}, axis = 1, inplace = True)
    usd_rub['date'] = pd.to_datetime(usd_rub['date'])
    final_df = merge_dfs(final_df, usd_rub)
    del usd_rub

    # effr
    effr = process_effr(path_total)
    effr['date'] = pd.to_datetime(effr['date'])
    final_df = merge_dfs(final_df, effr)
    del effr

    # S&P500
    df = pd.read_csv(os.path.join(path_total, r'macro\S&P500.csv'))
    df['date'] = pd.to_datetime(df['date'])
    df = df[['date', 'close', 'volume']]
    final_df = merge_dfs(final_df, df)
    del df

    # debt
    df = process_debt(path_total)
    df['date'] = pd.to_datetime(df['date'])
    final_df = merge_dfs(final_df, df)
    del df

    # GDP USA
    df = process_gdp_usa(path_total)
    df['date'] = pd.to_datetime(df['date'])
    final_df = merge_dfs(final_df, df)
    del df

    # Monetary supply
    df = process_macro_monthtly(path_total, 'monetary_agg_processed.csv')
    df['date'] = pd.to_datetime(df['date'])
    final_df = merge_dfs(final_df, df)
    del df

    # Households bm
    df = process_macro_monthtly(path_total, 'households_bm_processed.csv')
    df['date'] = pd.to_datetime(df['date'])
    final_df = merge_dfs(final_df, df)
    del df

    # GDP RU
    df = process_gdp_ru(path_total)
    df['date'] = pd.to_datetime(df['date'])
    final_df = merge_dfs(final_df, df)
    del df

    # imoex
    df = pd.read_csv(os.path.join(path_total, r'macro\imoex.csv'))
    df.rename({'Unnamed: 0':'date'}, axis = 1, inplace = True)
    df['date'] = pd.to_datetime(df['date'])
    final_df = merge_dfs(final_df, df)
    del df

    # interbanks rates
    df = pd.read_csv(os.path.join(path_total, r'macro\data_interbank_rates.csv'))
    df.rename({'Unnamed: 0':'date'}, axis = 1, inplace = True)
    df['date'] = pd.to_datetime(df['date'])
    final_df = merge_dfs(final_df, df)
    del df

    # precious metals
    df = pd.read_csv(os.path.join(PATH, r'macro\data_precious_metals.csv'))
    df.rename({'Unnamed: 0':'date'}, axis = 1, inplace = True)
    df['date'] = pd.to_datetime(df['date'])
    final_df = merge_dfs(final_df, df)
    del df
    
    
    return final_df

In [30]:
df = merge_stocks(os.path.join(PATH, 'renewable_energy'), PATH)
df.to_csv('renewable_energy.csv', index = False , encoding = 'utf-8')

In [31]:
df = merge_stocks(os.path.join(PATH, 'fintech'), PATH)
df.to_csv('fintech.csv', index = False , encoding = 'utf-8')

In [32]:
df = merge_stocks(os.path.join(PATH, 'industrial_goods'), PATH)
df.to_csv('industrial_goods.csv', index = False , encoding = 'utf-8')

In [33]:
df = merge_stocks(os.path.join(PATH, 'healthcare_services'), PATH)
df.to_csv('healthcare_services.csv', index = False , encoding = 'utf-8')